<a href="https://colab.research.google.com/github/cimbelli/LAU/blob/main/IT_1.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generate LAU file for Italy

In [25]:
### IMPORT LIBRARIES

import pandas as pd
import os, sys
import zipfile
import glob
import shutil
import numpy as np
!pip install wget
import wget

year = 2022

In [26]:
def extract(title, urlzipfile, cond, name):

    print('\n---', title, '---')
    zipfilename = os.path.basename(urlzipfile)
    out_dir = os.getcwd()
    outfile = ''
    if not os.path.isfile(zipfilename):
        #!wget $urlzipfile
        wget.download(urlzipfile,out = out_dir)
    if os.path.splitext(zipfilename)[-1] == '.zip':
        zip = zipfile.ZipFile(zipfilename)
        for file in zip.namelist():
            if file.endswith(cond) or file.endswith(cond[:-1]): # in the case of xls or xlsx files

                if not file:
                    continue
                source = zip.open(file)
                #target = open(os.path.basename(file), "wb")
                ext = os.path.splitext(cond)[-1]
                outfile = name + '_' + str(year) + ext
                target = open(outfile, "wb")
                with source, target:
                    shutil.copyfileobj(source, target)
        zip.close()
        os.remove(zipfilename)
    else:
        if title != prev_title:
            outfile = name + '_' + str(year) + os.path.splitext(zipfilename)[-1]
            os.replace(zipfilename, outfile)
        else:
            outfile = os.path.basename(zipfilename)
    print(outfile)
    return outfile

def openfile(filename, cols):
    ext = os.path.splitext(filename)[-1]
    if ext == '.xls' or ext == '.xlsx':
        df = pd.read_excel(filename, sheet_name=0, usecols = cols, dtype={'LAU CODE': object})
    if ext == '.csv':
        df = pd.read_csv(filename, sep=';', encoding="utf-8", quotechar='"',usecols = cols, skiprows=1, index_col=False,  dtype={population_code: object}).dropna()
    return df

def trim_all_columns(df):
    """
    Trim whitespace from ends of each value across all series in dataframe
    """
    trim_strings = lambda x: x.strip() if isinstance(x, str) else x
    return df.applymap(trim_strings)

In [36]:
### FILES URL and NAMES
yrange = ''
if year in range (2022, 2024):
    yrange = '2022-2023'
    yrange1 = '-2022-2023'
elif year in range (2017, 2022):
    yrange = '2017-2021'
    yrange1 = '_2017-2021'
else:
    print("year not found")

nomenclature_title = 'NOMENCLATURE'
nomenclature_zipfile = 'https://www.istat.it/storage/codici-unita-amministrative/Archivio-elenco-comuni-codici-e-denominazioni_Anni_' + yrange + '.zip'
nomenclature_cond = '31_12_' + str(year) + '.xlsx'
nomenclature_code = 'Codice Comune formato alfanumerico'
nomenclature_name = 'Denominazione in italiano'
nomenclature_nuts3 = 'Codice NUTS3 2021' if year > 2020 else 'Codice NUTS3 2010'
nomenclature_cols = [nomenclature_code,nomenclature_name,nomenclature_nuts3]

population_title = 'POPULATION'
population_zipfile = 'https://demo.istat.it/data/p2/P2_' + str(year-1) + '_it_Comuni.zip'
population_cond = 'P2_'  + str(year-1) + '_it_Comuni.csv'
population_code = 'Codice comune'
population_name = 'Comune'
population_1stJan = 'Popolazione al 1° gennaio - Totale'
population_cols = [population_code, population_name, population_1stJan]

classifications_title = 'CLASSIFICATIONS'
classifications_zipfile = 'https://www.istat.it/it/files/2015/04/Classificazioni-statistiche-Anni' + yrange1 + '.zip'
classifications_cond = '31_12_' + str(year) + '.xlsx'
classifications_code = 'Codice Istat del Comune \n(numerico)'
classifications_name = 'Denominazione (Italiana e straniera)'
classifications_name2 = 'Denominazione altra lingua'
classifications_area = 'Superficie territoriale (kmq) al 01/01/' + str(year)
classifications_coast = 'Zone costiere'
classifications_deg = 'Grado di urbanizzazione'
classifications_cols = [classifications_code,classifications_name,classifications_name2,classifications_area,classifications_coast,classifications_deg]

cityfua_title = 'CITYFUA'
cityfua_file = 'https://www.istat.it/it/files/2019/11/Elenco-city-e-composizione-FUA-31-12-2020.xlsx'
cityfua_cond = cityfua_file[-9:]
cityfua_code = 'Procom numerico'
cityfua_citycode = 'Codice City'
cityfua_cityname = 'Denominazione City'
cityfua_fuacode = 'Codice FUA'
cityfua_fuaname = 'Denominazione FUA'
cityfua_cols = [cityfua_code,cityfua_citycode,cityfua_cityname,cityfua_fuacode,cityfua_fuaname]

prev_title = "PREVIOUS LAU FILE"
#prev_fileurl = 'https://www.alessandrocimbelli.eu/gopa/NUTS_LAU_A_IT_' + str(year-1) + '_0000_V0001.zip'
prev_fileurl = 'https://github.com/cimbelli/LAU/raw/main/NUTS_LAU_A_IT_' + str(year-1) + '_0000_V0001.xlsx'
prev_cond = '.xlsx'
prev_nuts3 = 'NUTS 3 CODE'
prev_laucode = 'LAU CODE'
prev_name = 'LAU NAME NATIONAL'
prev_area = 'TOTAL AREA (m2)'
prev_cols = [prev_nuts3, prev_laucode, prev_name, prev_area]

lau_code = "LAU CODE"
change = "CHANGE (Y/N)"
prev_flag = False

# field mapping
columns={nomenclature_nuts3: "NUTS 3 CODE",
         lau_code: "LAU CODE",
         population_name: "LAU NAME NATIONAL",
         classifications_name2: "LAU NAME LATIN",
         change: "CHANGE (Y/N)",
         population_1stJan: "POPULATION",
         classifications_area: "TOTAL AREA (m2)",
         classifications_deg: "DEGURBA",
         classifications_coast: "COASTAL AREA (yes/no)",
         cityfua_citycode: "CITY_ID",
         cityfua_cityname: "CITY_NAME",
         cityfua_fuacode: "FUA_ID",
         cityfua_fuaname: "FUA_NAME"
         }

country ='IT'
outname = 'NUTS_LAU_A_IT_' + str(year) + '_0000_V0001.xlsx'

In [37]:
print(nomenclature_zipfile)
print(population_zipfile)
print(classifications_zipfile)
print(cityfua_file)
print(prev_fileurl)

https://www.istat.it/storage/codici-unita-amministrative/Archivio-elenco-comuni-codici-e-denominazioni_Anni_2022-2023.zip
https://demo.istat.it/data/p2/P2_2021_it_Comuni.zip
https://www.istat.it/it/files/2015/04/Classificazioni-statistiche-Anni-2022-2023.zip
https://www.istat.it/it/files/2019/11/Elenco-city-e-composizione-FUA-31-12-2020.xlsx
https://github.com/cimbelli/LAU/raw/main/NUTS_LAU_A_IT_2021_0000_V0001.xlsx


In [38]:
### FILES DOWNLOAD and RENAME

#%%capture test
print('DOWNLOAD and UNZIP')
nomenclature_file = extract(nomenclature_title, nomenclature_zipfile, nomenclature_cond,'nomenclature')
population_file = extract(population_title, population_zipfile, population_cond,'population')
classifications_file = extract(classifications_title, classifications_zipfile, classifications_cond,'classifications')
cityfua_file = extract(cityfua_title, cityfua_file, cityfua_cond,'cityfua')
prev_file = extract(prev_title, prev_fileurl, prev_cond,'prev')

DOWNLOAD and UNZIP

--- NOMENCLATURE ---
nomenclature_2022.xlsx

--- POPULATION ---
population_2022.csv

--- CLASSIFICATIONS ---
classifications_2022.xlsx

--- CITYFUA ---
cityfua_2022.xlsx

--- PREVIOUS LAU FILE ---
NUTS_LAU_A_IT_2021_0000_V0001.xlsx


In [39]:
### FILES OPEN
nomenclature = openfile(nomenclature_file, nomenclature_cols)
population = openfile(population_file, population_cols)
classifications = openfile(classifications_file,classifications_cols)
cityfua = openfile(cityfua_file, cityfua_cols)

### CLEAN RECORDS FROM BLANKS
nomenclature = trim_all_columns(nomenclature)
population = trim_all_columns(population)
classifications = trim_all_columns(classifications)
cityfua = trim_all_columns(cityfua)

# check LAU file of the previous year

if os.path.exists(prev_file):
    prev_flag = True
    previous = openfile(prev_file, prev_cols)
    previous = trim_all_columns(previous)
    previous = previous.rename(columns={prev_nuts3: "nuts3", prev_laucode: "lau", prev_name: "name", prev_area: "area"})

population[lau_code] = population[population_code].astype(str)
population[population_code] = pd.to_numeric(population[population_code])
population[population_1stJan] = population[population_1stJan].astype(int)

In [40]:
### 1st JOIN
df = pd.merge(nomenclature,population,left_on=nomenclature_code, right_on=population_code)
df = df[[nomenclature_nuts3,lau_code, population_code, population_name, population_1stJan]]

### 2nd JOIN
df1 = pd.merge(df,classifications,left_on=population_code, right_on=classifications_code)
df1 = df1[[nomenclature_nuts3,lau_code, population_code, population_name,classifications_name2, population_1stJan, classifications_area,
           classifications_deg,classifications_coast]]

### 3rd JOIN
df2 = pd.merge(df1,cityfua,left_on=population_code, how='left', right_on=cityfua_code)
df2[change]= ''
df2 = df2[[nomenclature_nuts3,lau_code, population_name,classifications_name2, change, population_1stJan, classifications_area,classifications_deg,
           classifications_coast, cityfua_citycode,cityfua_cityname,cityfua_fuacode,cityfua_fuaname]]

# some value adjustments
df2 = df2.replace('N.d.', 0)
df2[classifications_area] = df2[classifications_area].astype(float)*1000000
df2[classifications_area] = df2[classifications_area].astype(int)

df2.loc[df2[classifications_coast] == 1, classifications_coast] = 'yes'
df2.loc[df2[classifications_coast] == 0, classifications_coast] = 'no'

df2 = df2.fillna('')
df2.replace(np.nan,'',regex=True)
df2 = df2.replace(0,'')

### 4th JOIN if previous file exists
if prev_flag:
    df3 = pd.merge(df2, previous, left_on=lau_code, how='left', right_on="lau")
    df3[change]= 'N'
    df3.loc[df3[nomenclature_nuts3] != df3['nuts3'], change] = 'Y'
    df3.loc[df3[population_name] != df3['name'], change] = 'Y'
    #df3.loc[df3[classifications_area] != df3['area'], change] = 'Y'
    df2 = df3[[nomenclature_nuts3,lau_code, population_name,classifications_name2, change, population_1stJan, classifications_area,classifications_deg,
           classifications_coast, cityfua_citycode,cityfua_cityname,cityfua_fuacode,cityfua_fuaname]]


# output
out = df2.rename(columns = columns)
out.to_excel(outname, sheet_name = country, index=False)

In [41]:
out.head()

,NUTS 3 CODE,LAU CODE,LAU NAME NATIONAL,LAU NAME LATIN,CHANGE (Y/N),POPULATION,TOTAL AREA (m2),DEGURBA,COASTAL AREA (yes/no),CITY_ID,CITY_NAME,FUA_ID,FUA_NAME
0,ITC11,001001,Agliè,,N,2545,13146300,2,no,,,,
1,ITC11,001002,Airasca,,N,3633,15739300,3,no,,,IT004F,Torino
2,ITC11,001003,Ala di Stura,,N,459,46331600,3,no,,,,
3,ITC11,001004,Albiano d'Ivrea,,N,1638,11731399,3,no,,,,
4,ITC11,001006,Almese,,N,6355,17875700,2,no,,,IT004F,Torino
